In [2]:
pip install together 

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------- ---------------- 1.6/2.7 MB 8.3 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 8.6 MB/s eta 0:00:00
  Attempting uninstall: pillow
    Found existing installation: pillow 10.4.0
    Uninstalling pillow-10.4.0:
      Successfully uninstalled pillow-10.4.0
  Attempting uninstall: rich
    Found existing installation: rich 13.7.1
    Uninstalling rich-13.7.1:
      Successfully uninstalled rich-13.7.1
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
streamlit 1.37.1 requires pillow<11,>=7.1.0, but you have pillow 11.2.1 which is incompatible.


In [61]:
pip install pyvis

   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/756.0 kB ? eta -:--:--
   ------------- -------------------------- 262.1/756.0 kB ? eta -:--:--
   ------------- -------------------------- 262.1/756.0 kB ? eta -:--:--
   ------------- -------------------------- 262.1/756.0 kB ? eta -:--:--
   -------------------------------------- 756.0/756.0 kB 790.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
from together import Together
from neo4j import GraphDatabase
import json
from pyvis.network import Network

In [ ]:
# === HARDCODED CONFIG ===
TOGETHER_API_KEY = "api-key"
NEO4J_URI = "bolt://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "password"

In [21]:
# === TOGETHER CLIENT ===
# Initialize Together client
client = Together(api_key=TOGETHER_API_KEY)

# === SYSTEM PROMPT FOR CYHPER CONVERSION ===
# Define the system prompt to instruct the model
system_prompt = (
    "You are an expert Cypher query generator for Neo4j graph databases. Convert natural language requests into syntactically correct and optimized Cypher queries. Return only the Cypher code—no explanations, no formatting, no commentary."
    "Translate the following natural language instruction into a Cypher query. Output only the Cypher query, without any additional explanation or formatting."
    "for a Neo4j graph where each entry is a triplet:\n"
    "- subject (Entity node with name)\n"
    "- relation (relationship type, e.g., HAS_PATHOLOGICAL_FEATURE, has_cause, associated_with, treated_by, has_symptom, etc)\n"
    "- object (Entity node with name)\n\n"
    "Generate Cypher queries that strictly follow this schema:\n"
    "(:Entity {name: '<subject>'})-[:<RELATION>]->(:Entity {name: '<object>'})\n\n"
    "Return only the Cypher query, no explanations or comments."
    "Note: The database follows a strict naming convention—disease names begin with a capitalized first letter, followed by either 'disease' or 'syndrome."
    """ sample query for your reference. 
    MATCH (s:Entity)-[r]->(o:Entity)
    WHERE s.name CONTAINS 'Aicardi' OR o.name CONTAINS 'Aicardi'
    RETURN s.name, type(r), o.name
    Refer this query to generate cypher queries """
)

In [23]:
# === CONVERT NL TO CYPHER ===
def nl_to_cypher(nl_query):
    response = client.chat.completions.create(
        model="meta-llama/Llama-4-Maverick-17B-128E-Instruct-FP8",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": nl_query}
        ]
    )
    return response.choices[0].message.content.strip()

In [25]:
# === MAIN ===
if __name__ == "__main__":
    nl_query = input("Enter a natural language query: ")
    cypher = nl_to_cypher(nl_query)
    print(f"\nGenerated Cypher:\n{cypher}\n")

Enter a natural language query:  provide all information about fabry disease



Generated Cypher:
MATCH (s:Entity)-[r]->(o:Entity) 
WHERE s.name CONTAINS 'Fabry disease' OR o.name CONTAINS 'Fabry disease' 
RETURN s.name, type(r), o.name



In [27]:
# Edge color map
relation_colors = {
    "treated_by": "#00cc66",
    "has_symptom": "#ff6600",
    "caused_by_deficiency_of": "#cc0000",
    "increases_activity_of": "#6600cc",
    "has_cause": "#006699",
    "default": "#999999"
}

net = Network(
    notebook=False,
    height="800px",
    width="100%",
    bgcolor="#ffffff",
    font_color="black",
    directed=True
)
net.heading = "Knowledge Graph"
net.barnes_hut(
    gravity=-20000,
    central_gravity=0.3,
    spring_length=150,
    spring_strength=0.05,
    damping=0.95,
    overlap=0.8
)
net.show_buttons(filter_=["physics"])

driver = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "0987654321"))

with driver.session() as session:
    query = cypher
    result = session.run(query)
    added_nodes = set()

    for record in result:
        source = record["s.name"]
        relation = record["type(r)"]
        target = record["o.name"]

        if source not in added_nodes:
            net.add_node(source, label=source, color="#82caff", shape="dot", size=15)
            added_nodes.add(source)
        if target not in added_nodes:
            net.add_node(target, label=target, color="#f0a1a8", shape="dot", size=15)
            added_nodes.add(target)

        color = relation_colors.get(relation.lower(), relation_colors["default"])
        net.add_edge(source, target, label=relation, color=color, width=2, arrows="to", smooth=True)

net.save_graph(r"C:\AISSMS-IOIT-Practicals\Mavericks_AMUHACKS4.0\Graphs\graph.html")
print("knowledge graph saved as graph.html!")

knowledge graph saved as graph.html!
